In [1]:
# cell-1  
#load and clean the data (removing diacritics and unwanted text)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, BertForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


log_file = 'MNADv2.txt'
with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')

df = pd.read_csv('MNADv2/MNADv2.csv')
df.fillna('', inplace=True)


df['Category'] = df['Category'].astype('category')
# display(dfc['meter'].unique())

df['label'] = df['Category'].cat.codes #assign cat_value for each meter type
# dftrain, dftest = train_test_split(df, test_size=0.20, random_state=42, stratify=df['label'])
# ytrain = dftrain['label'].values.tolist()
# ytest = dftest['label'].values.tolist()



max_sequence_length = 128
train_batch_size = 128
classes_num = len(df['Category'].unique())
df = df[['Body', 'label']]
display(classes_num)
# display(len(df))
# display(len(dftrain))
# display(len(dftest))

# dftrain = dftrain[['Body']]
# dftest = dftest[['Body']]
# df = ''

# display(df[:2])

ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.2)



models = ['faisalq/bert-base-arabic-wordpiece', 'faisalq/bert-base-arabic-senpiece',
          'faisalq/bert-base-arabic-bbpe']


for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
              
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
        dataset_train = ds['train']
        dataset_validation = ds['test']                                                     
        
      

        def preprocess_function(examples):
            return tokenizer(examples['Body'], truncation=True, padding="max_length",
                            max_length=max_sequence_length)
        
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)
        
       
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}


        
        
        epochs = 7
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 256
        
        training_args = TrainingArguments(
            output_dir = 'bert/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 1, #only save the last 5 checkpoints
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            logging_steps = 1000, #50_000
            evaluation_strategy = 'steps',
            # evaluate_during_training = True,
            eval_steps = 1000
            
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            # data_collator=data_collator,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics = compute_metrics
        )
        
        
        # trainer.train(resume_from_checkpoint=True)
        trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results.to_csv('MNADv2_results.csv')
display(best_results)

2024-02-12 17:12:26.765973: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-12 17:12:26.791231: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-12 17:12:27.154854: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


20

faisalq/bert-base-arabic-wordpiece, try:0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-wordpiece and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/855591 [00:00<?, ? examples/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-bbpe and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/855591 [00:00<?, ? examples/s]

Map:   0%|          | 0/213898 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
1000,0.691000,0.590281,0.791298,0.682653
2000,0.579200,0.543536,0.804411,0.709508
3000,0.548700,0.527571,0.808077,0.725470
4000,0.492900,0.518878,0.812939,0.732112
5000,0.470200,0.512700,0.815169,0.740451
6000,0.469200,0.505807,0.816478,0.740602
7000,0.436600,0.524987,0.814295,0.741304
8000,0.378600,0.530692,0.814052,0.742296
9000,0.383300,0.528067,0.814659,0.741613
10000,0.382700,0.526882,0.816244,0.746100


,Model,Accuracy,F1
0,faisalq/bert-base-arabic-bbpe,0.816244,0.746100
1,faisalq/bert-base-arabic-bbpe,0.816244,0.746100
2,faisalq/bert-base-arabic-bbpe,0.816244,0.746100
3,faisalq/bert-base-arabic-senpiece,0.810690,0.731307
4,faisalq/bert-base-arabic-senpiece,0.810690,0.731307
5,faisalq/bert-base-arabic-senpiece,0.810690,0.731307
6,faisalq/bert-base-arabic-wordpiece,0.809802,0.734367
7,faisalq/bert-base-arabic-wordpiece,0.809802,0.734367
